In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/en

In [2]:
content_df = pd.read_csv('../data/Train_Dataset.csv')
label_df = pd.read_csv('../data/Train_Dataset_Label.csv')
train_df = content_df.merge(label_df, on='id')
train_df = train_df.fillna('EMPTY')

In [3]:
test_df = pd.read_csv('../data/Test_Dataset.csv')
test_df = test_df.fillna('EMPTY')
test_title_data = pd.read_csv('./test_title_word_vector.csv', header=None).values
test_content_data = pd.read_csv('./test_content_word_vector.csv', header=None).values
X_test = np.concatenate((test_title_data, test_content_data), axis=1)

In [4]:
y = train_df['label'].values
train_title_data = pd.read_csv('./train_title_word_vector.csv', header=None).values
train_content_data = pd.read_csv('./train_content_word_vector.csv', header=None).values
X = np.concatenate((train_title_data, train_content_data), axis=1)

from sklearn.preprocessing import LabelEncoder
import keras

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = keras.utils.np_utils.to_categorical(y)

Using TensorFlow backend.


In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(5, shuffle=True, random_state=2019)

model_list = []
score_list = []
for index, (train_index, valid_index) in enumerate(kf.split(X, y)):
    print(index)
    X_train, X_valid, y_train, y_valid = X[train_index], X[valid_index], y[train_index], y[valid_index]
    
    model = tf.keras.Sequential([
                tf.keras.layers.Dense(32, activation=tf.nn.relu, input_shape=(768*2,)),
                tf.keras.layers.Dense(32, activation=tf.nn.relu),
                tf.keras.layers.Dense(3, activation='softmax')
            ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), callbacks=[early_stop], batch_size=32, epochs=20, verbose=0)
    y_pred = model.predict(X_valid)
    y_valid = np.argmax(y_valid, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    valid_score = f1_score(y_valid, y_pred, average='macro')
    model_list.append(model)
    score_list.append(valid_score)
print(np.mean(score_list))

0
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
1
2
3
4
0.7619517665907752


In [6]:
y_pred = None
for model in model_list:
    if y_pred is None:
        y_pred = model.predict(X_test)
    else:
        y_pred = y_pred + model.predict(X_test)
y_pred = np.argmax(y_pred / 5, axis=1)

In [7]:
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id'].values
submission_df['label'] = y_pred
submission_df.to_csv('./submission.csv', index=False)

In [8]:
fine_tune_submission_df = pd.read_csv('/Users/yitinglin/Desktop/submission.csv')
fine_tune_label = fine_tune_submission_df['label'].values + 1
feature_extract_label = submission_df['label'].values
label = []
for i in range(len(feature_extract_label)):
    if feature_extract_label[i] == 0:
        label.append(feature_extract_label[i])
        continue
    if feature_extract_label[i] != fine_tune_label[i]:
        label.append(fine_tune_label[i])
    else:
        label.append(feature_extract_label[i])
submission_df['label'] = label

In [9]:
submission_df.to_csv('./submission.csv', index=False)